In [45]:
import sys
# sys.path.append(r"Y:\Users\Yingyue\Gates_Lab_Suite-master")
sys.path.append(r"/Volumes/funkflower/Users/Yingyue/Gates_Lab_Suite-master")
from Core_Definition import *
from Auto_Algorithm import *
from Visualization import *
import os
from SPAM import *
import numpy as np
import xlsxwriter as xlsx
from scipy import optimize
import random
import math
from math import floor
import pickle
import uuid

In [46]:
import json, pickle
import qiskit
#transpiler reducinig qiskit circuit to QASM strings
from qiskit import transpile
# Use AerSimulator
from qiskit_aer import AerSimulator
from qiskit import IBMQ, QuantumCircuit, execute
from qiskit.providers.ibmq.managed import IBMQJobManager
from qiskit.extensions import UnitaryGate
from qiskit.quantum_info import Pauli, Clifford, Statevector,Operator
from qiskit.visualization import array_to_latex
from scipy.stats import sem, unitary_group
from scipy.linalg import sqrtm,expm
import qiskit.quantum_info as qi
from qiskit.quantum_info import Pauli
import pylatexenc

In [65]:
def state_init_bell_pairs(qc,n):
    for i in range (n):
        qc.Add_Gate(Quantum_Gate("HAD",i))
    for i in range (n):
        qc.Add_Gate(Quantum_Gate("CNOT",i,int(i+n)))

        
#should we set_mapping in this function? need to think about if it takes mapping into account   
def add_pauli_twirl(qc,n):
    pauliLayer = [random.choice(['I','X','Y','Z']) for j in range(n)]
    q_index=0
    for pauli in pauliLayer :
        if (pauli=="I"):
            qc.Add_Gate(Quantum_Gate("SKAX",2*n,angle=1))
        elif (pauli=="X"):
            qc.Add_Gate(Quantum_Gate("SKAX",q_index,angle=1))
        elif (pauli=="Y"):
            qc.Add_Gate(Quantum_Gate("SKAY",q_index,angle=1))
        elif (pauli=="Z"):
            qc.Add_Gate(Quantum_Gate("AZ",q_index,angle=1))
        q_index+=1
    return pauliLayer

def add_pauli_twirl_wSE(qc,n):
    pauliLayer = [random.choice(['I','X','Y','Z']) for j in range(n)]
    q_index=0
    for pauli in pauliLayer :
        if (pauli=="I"):
            qc.Add_Gate(Quantum_Gate("SKAX",2*n,angle=1))
        elif (pauli=="X"):
            qc.Add_Gate(Quantum_Gate("SKAX",q_index,angle=1))
        elif (pauli=="Y"):
            qc.Add_Gate(Quantum_Gate("SKAY",q_index,angle=1))
        elif (pauli=="Z"):
            qc.Add_Gate(Quantum_Gate("AZ",q_index,angle=1))
        
        if (q_index==(int(n/2)-1)):
            for i in range (n):
                qc.Add_Gate(Quantum_Gate("SKAX",n+i,angle=1))
        q_index+=1
    return pauliLayer
    
def add_clifford_layer(qc,n,clifford):
    if (clifford=="CNOT"):
        for i in range (floor(n/2)):
            qc.Add_Gate(Quantum_Gate("CNOT",2*i,2*i+1))
            
def bell_measurement(qc,n):
    for i in range (n-1,-1,-1):
        qc.Add_Gate(Quantum_Gate("SKAX",int(i+n),angle=1))
        qc.Add_Gate(Quantum_Gate("CNOT",i,int(i+n)))
    for i in range (n-1,-1,-1):
        qc.Add_Gate(Quantum_Gate("HAD",i))

def map_statepop_2_ibm_mapping(counts,n):
    counts_ibm_mapping=[0 for i in range (2**(2*n))]
    idx_ibm_mapping=[]
    counts_ibm_mapping_dic={}
    if len(str((2*n)))<2:
        f="00"+str(2*n)+"b"
    elif len(str((2*n)))<3:
        f="0"+str(2*n)+"b"
    else:
        raise ValueError("n is too big")
    for gates_idx in range (2**(2*n)):
        gates_idx_str=format(gates_idx,f)
        ibm_idx=0
        for i in range (2*n):
            ibm_idx+=2**(i)*int(gates_idx_str[i])
        ibm_idx_str=format(ibm_idx,f)
#         idx_ibm_mapping.append(ibm_idx_str)
        counts_ibm_mapping[ibm_idx]=counts[gates_idx]
        counts_ibm_mapping_dic[ibm_idx_str]=counts[gates_idx]
    return counts_ibm_mapping, counts_ibm_mapping_dic
        
        
#for qiskit
def prepare_bell_state_1q(circuit,index1,index2):
	circuit.h([index1])
	circuit.cx([index1],[index2])
        
def bell_measurement_1q(circuit,index1,index2): 
	# info qubit at index1
	circuit.cx([index1],[index2])
	circuit.h([index1])

def pauli_gate_1q(circuit,index,pauli=None): #For stabilizer simulator to work, cannot use Pauli class
	if pauli == 'I':
		circuit.id([index])
	elif pauli == 'Z':
		circuit.z([index])
	elif pauli == 'X':
		circuit.x([index])
	elif pauli == 'Y':
		circuit.y([index])
	else:
		assert 1==0

    
    
    

In [66]:
## Additional functions

def int_to_pauli(i,n):
    p = np.base_repr(i,base=4)
    p = '0'*(n-len(p)) + p
    p = p.replace('0','I').replace('1', 'X').replace('2', 'Y').replace('3', 'Z')
    return p

def commute(p,q):
    c = 1
    n = len(p)
    for i in range(n):
        if p[i] != 'I' and q[i] != 'I':
            if p[i] != q[i]:
                c *= -1
    return c

def fidelity_to_error(pauli_fidelity,n):
    N = 4**n
    pauli_error = {}
    for i in range(N):
        p = int_to_pauli(i,n)
        pauli_error[p] = 0
        for j in range(N):
            q = int_to_pauli(j,n)
            pauli_error[p] += pauli_fidelity[q] * commute(p,q) / N
    return pauli_error
    

In [76]:
Set_Mapping([1,2,4,5,3])
nqubit=2
# ancillae=2
S=Quantum_Circuit(2*nqubit+1,"qc")

# depth=[2**x for x in range(1,6)] #depth for Concatenation
depth=[2,4,8]
C=10 #number of samples per depth
all_circuits=[]#a list of lists. circuits for all depths. each element is a list of circuit of a certain depth d. 
circuits=[] #a list of circuits with a certain depth d
eab_data=[] #a list of dictionaries. Each dictionary is the results and relevant info of one circuit
results={}
clifford_layer="CNOT"
#for qiskit 
backend = AerSimulator()
t_echo=0
stablizer_save=[]
filename = str(uuid.uuid4())

for d in depth:
    t_echo=d/2
    print ("\nDepth = %d"%d)
    print ("Number of samples = %d"%C)
    for i in range (C):
        S.depth=0
        S.gates=[]
#         qiskit_circ=QuantumCircuit(2*nqubit,2*nqubit)
        qiskit_c_state = QuantumCircuit(2*nqubit,2*nqubit)
        qiskit_c_gates = QuantumCircuit(nqubit)

        #gates circuit state prep
        state_init_bell_pairs(S,nqubit)
        #qiskit circuit state prep
        for j in range(nqubit):
            prepare_bell_state_1q(qiskit_c_state,j,j+nqubit)
        qiskit_c_state.barrier()

        
        for j in range(d):
            if (j==t_echo):
                layer=add_pauli_twirl_wSE(S,nqubit)
            #gates circuit add pauli layer
            else:
                layer=add_pauli_twirl(S,nqubit)
            # print (layer)
            
            #qiskit circuit add pauli layer
            for j in range(nqubit):
                pauli_gate_1q(qiskit_c_gates,j,pauli=layer[j])


            #gates circuit add clifford layer
            add_clifford_layer(S,nqubit,clifford_layer)
            #attempted to simulated noise but it's wrong. This doesn't add noise from shot to shot.
#             xx_err=random.randrange(0, 1000, 1)
#             for q in range(nqubit):
#                 if (xx_err==q):
#                     S.Add_Gate(Quantum_Gate("SKAX",q,angle=1))
                
            
            #qiskit circuit add clifford layer
            if clifford_layer == 'Id':
                pass
            elif clifford_layer == 'CNOT':
                ngates = int(nqubit/2)
                for j in range(ngates):
                    qiskit_c_gates.cx(2*j,2*j+1)
            qiskit_c_gates.barrier()
            
        #gates circuit add Pauli twirl
        layer=add_pauli_twirl(S,nqubit)
        
        #qiskit circuit add Pauli twirl
        for j in range(nqubit):
            pauli_gate_1q(qiskit_c_gates,j,pauli=layer[j])
        #save the stablizer representation
        cliffordOp = Clifford(qiskit_c_gates)
        
        #for qiskit: append gates to state
        qiskit_circ = qiskit_c_state.compose(qiskit_c_gates,range(nqubit))
        
        #gates circuti bell measurement
        bell_measurement(S,nqubit)
        #qiskit circuti bell measurement
        for j in range(nqubit):
            bell_measurement_1q(qiskit_circ,j,j+nqubit)
        qiskit_circ.barrier(range(2*nqubit))
        qiskit_circ.measure(range(2*nqubit),range(2*nqubit))
        #for qiskit 
        qc_compiled = transpile(qiskit_circ, backend)
#         print (qc_compiled)
        job_sim = backend.run(qc_compiled, shots=2000)
        result_sim = job_sim.result()
        counts_qiskit = result_sim.get_counts(qc_compiled)


        circuits.append(S.GatesLab_Sequence())
        print (S.GatesLab_Sequence())
        #package and format results
        results["n"]=nqubit
        results["L"]=d
        results["clifford"] = cliffordOp.to_dict()
        stablizer_save.append(cliffordOp.to_dict())
        results["clifford_layer"] = clifford_layer
        results["circuit"]=S.GatesLab_Sequence()
        
        statef=S.Simulate()
        statef_traced=Trace_Out(statef,[0,1,2,3])
        counts=[round(2000*x) for x in  statef_traced.population]
        
        counts_ibm_mapping, counts_ibm_mapping_dic=map_statepop_2_ibm_mapping(counts,nqubit)
        results["counts"]=counts_ibm_mapping_dic
        eab_data.append(results)
#         if i==0:
#             print (results)
#             print(counts_qiskit)
# #             print (cliffordOp)
        results={}
        
    all_circuits.append(circuits)
    
    circuits=[]
# with open("/Volumes/funkflower/Users/Yingyue/Gates_Lab_Suite-master/PauliNoiseEstimation/data/"+filename+"12453withpadding", "wb") as fp:
#     pickle.dump(eab_data, fp)

# >>> with open("test", "rb") as fp:   # Unpickling
# ...   b = pickle.load(fp)
#     print (all_circuits)


Depth = 2
Number of samples = 10
HAD1:HAD2:CNOT14:CNOT25:SKAX1+1.0000:SKAX2+1.0000:CNOT12:AZ1+1.0000:SKAX4+1.0000:SKAX5+1.0000:SKAY2+1.0000:CNOT12:SKAY1+1.0000:SKAX2+1.0000:SKAX5+1.0000:CNOT25:SKAX4+1.0000:CNOT14:HAD2:HAD1
HAD1:HAD2:CNOT14:CNOT25:SKAY1+1.0000:SKAX2+1.0000:CNOT12:SKAX1+1.0000:SKAX4+1.0000:SKAX5+1.0000:AZ2+1.0000:CNOT12:AZ1+1.0000:SKAX3+1.0000:SKAX5+1.0000:CNOT25:SKAX4+1.0000:CNOT14:HAD2:HAD1
HAD1:HAD2:CNOT14:CNOT25:SKAY1+1.0000:AZ2+1.0000:CNOT12:SKAX1+1.0000:SKAX4+1.0000:SKAX5+1.0000:AZ2+1.0000:CNOT12:SKAY1+1.0000:SKAX2+1.0000:SKAX5+1.0000:CNOT25:SKAX4+1.0000:CNOT14:HAD2:HAD1
HAD1:HAD2:CNOT14:CNOT25:SKAY1+1.0000:AZ2+1.0000:CNOT12:SKAX3+1.0000:SKAX4+1.0000:SKAX5+1.0000:SKAX2+1.0000:CNOT12:AZ1+1.0000:SKAX2+1.0000:SKAX5+1.0000:CNOT25:SKAX4+1.0000:CNOT14:HAD2:HAD1
HAD1:HAD2:CNOT14:CNOT25:AZ1+1.0000:SKAX2+1.0000:CNOT12:SKAY1+1.0000:SKAX4+1.0000:SKAX5+1.0000:SKAY2+1.0000:CNOT12:AZ1+1.0000:SKAY2+1.0000:SKAX5+1.0000:CNOT25:SKAX4+1.0000:CNOT14:HAD2:HAD1
HAD1:HAD2:CNOT14:CNOT25:

In [71]:
sys.path.append(r"/Volumes/funkflower/Users/Yingyue/Gates_Lab_Suite-master/PauliNoiseEstimation")
import EAB_process_modified 
import itertools
'''Specify a set of Pauli you want to estimate'''
pauli_request_list = [''.join(s) for s in itertools.product(['I','X','Y','Z'], repeat = nqubit)] #full

#process_EAB(Lrange, cb_data, pauli_request_list)
eab_result=EAB_process_modified.process_EAB(depth, eab_data, pauli_request_list)

print (eab_result)
print (stablizer_save)

circuit count: 0
circuit count: 1
circuit count: 2
circuit count: 3
circuit count: 4
circuit count: 5
circuit count: 6
circuit count: 7
circuit count: 8
circuit count: 9
circuit count: 10
circuit count: 11
circuit count: 12
circuit count: 13
circuit count: 14
circuit count: 15
circuit count: 16
circuit count: 17
circuit count: 18
circuit count: 19
circuit count: 20
circuit count: 21
circuit count: 22
circuit count: 23
circuit count: 24
circuit count: 25
circuit count: 26
circuit count: 27
circuit count: 28
circuit count: 29
{'fidelity_list': {'II': {2: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 4: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 8: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]}, 'IX': {2: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 4: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 8: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]}, 'IY': {2: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 4: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,

In [63]:
with open("/Volumes/funkflower/Users/Yingyue/Gates_Lab_Suite-master/PauliNoiseEstimation/data/"+filename, "rb") as fp:   # Unpickling
    b = pickle.load(fp)
print (filename)
print (b)
    

97c7b2c8-9d6e-473a-b28a-a67915b99307
[{'n': 2, 'L': 2, 'clifford': {'stabilizer': ['+IZ', '+ZI'], 'destabilizer': ['+IX', '-XI']}, 'clifford_layer': 'CNOT', 'circuit': 'HAD1:HAD2:CNOT13:CNOT24:SKAX1+1.0000:AZ2+1.0000:CNOT12:SKAX1+1.0000:SKAX3+1.0000:SKAX4+1.0000:SKAX2+1.0000:CNOT12:SKAX4+1.0000:CNOT24:SKAX3+1.0000:CNOT13:HAD2:HAD1', 'counts': {'0000': 0, '1000': 0, '0100': 0, '1100': 0, '0010': 2000, '1010': 0, '0110': 0, '1110': 0, '0001': 0, '1001': 0, '0101': 0, '1101': 0, '0011': 0, '1011': 0, '0111': 0, '1111': 0}}, {'n': 2, 'L': 2, 'clifford': {'stabilizer': ['+IZ', '-ZI'], 'destabilizer': ['-IX', '+XI']}, 'clifford_layer': 'CNOT', 'circuit': 'HAD1:HAD2:CNOT13:CNOT24:AZ1+1.0000:CNOT12:SKAX3+1.0000:SKAX4+1.0000:CNOT12:SKAX2+1.0000:SKAX4+1.0000:CNOT24:SKAX3+1.0000:CNOT13:HAD2:HAD1', 'counts': {'0000': 0, '1000': 0, '0100': 0, '1100': 0, '0010': 0, '1010': 0, '0110': 0, '1110': 0, '0001': 0, '1001': 2000, '0101': 0, '1101': 0, '0011': 0, '1011': 0, '0111': 0, '1111': 0}}, {'n': 2, '

In [20]:
raw_fidelity_list = eab_result["fidelity_list"]


# sys.exit(0)

fidelity_list = {}
stdev_list = {}

for pauli_label in pauli_request_list:
    if(pauli_label == 'I'*nqubit):
        fidelity_list[pauli_label] = 1.0
        stdev_list[pauli_label] = 0.0
    else:
        alpha, alpha_err = EAB_process_modified.fit_EAB(depth, raw_fidelity_list[pauli_label])
        fidelity_list[pauli_label] = alpha
        stdev_list[pauli_label] = alpha_err

# print(fidelity_list)

# print(stdev_list)
print("Parameters: n = %d, C = %d, " % (nqubit,C), "L = ", str(depth))
use_density_matrix=False
use_readout_error =False
shots=2000
if use_density_matrix:
    print("Density matrix based simulation")
else:
    print("Measurement based simulation, shots = %d" % shots)

if use_readout_error is True:
    print("Measurement bitflip rate = %f" % eps_readout)
else:
    print("No readout error")

# Average fidelity
print("Total error = ", 1-np.mean(list(fidelity_list.values())))

print("Label / Pauli infidelity / Standard deviation")
for pauli_label in pauli_request_list:
    print(pauli_label[::-1], 1-fidelity_list[pauli_label], stdev_list[pauli_label])
# print('Effective noise rate = ' + str(1-np.average(list(fidelity_list.values()))))



Parameters: n = 2, C = 10,  L =  [2, 4, 8, 16, 32]
Measurement based simulation, shots = 2000
No readout error
Total error =  0.0
Label / Pauli infidelity / Standard deviation
II 0.0 0.0
XI 0.0 inf
YI 0.0 inf
ZI 0.0 inf
IX 0.0 inf
XX 0.0 inf
YX 0.0 inf
ZX 0.0 inf
IY 0.0 inf
XY 0.0 inf
YY 0.0 inf
ZY 0.0 inf
IZ 0.0 inf
XZ 0.0 inf
YZ 0.0 inf
ZZ 0.0 inf


/opt/anaconda3/envs/snowflakes/lib/python3.8/site-packages/scipy/optimize/_minpack_py.py:803: RuntimeWarning: divide by zero encountered in divide
  transform = 1.0 / sigma
/opt/anaconda3/envs/snowflakes/lib/python3.8/site-packages/scipy/optimize/_minpack_py.py:488: RuntimeWarning: invalid value encountered in multiply
  return transform * (func(xdata, *params) - ydata)
/opt/anaconda3/envs/snowflakes/lib/python3.8/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


In [21]:
error_list = fidelity_to_error(fidelity_list,nqubit)
print("Label / Pauli error rates")
for pauli_label in pauli_request_list:
    print(pauli_label[::-1], error_list[pauli_label])

Label / Pauli error rates
II 1.0
XI 0.0
YI 0.0
ZI 0.0
IX 0.0
XX 0.0
YX 0.0
ZX 0.0
IY 0.0
XY 0.0
YY 0.0
ZY 0.0
IZ 0.0
XZ 0.0
YZ 0.0
ZZ 0.0


In [13]:
# True value for this specific noise model
# eps=0
# def f_true(P):
#     ans = 0.0
#     for Pi in P:
#         if Pi == 'Z' or Pi == 'Y':
#             ans += 2*eps
#     return ans
# def p_true(P):
#     if P == "XI" or P == "IX":
#         return eps
#     else:
#         return 0.0

# print("Label / True infidelity/ True Pauli error")
# for P in pauli_request_list:
#     print(P[::-1]," ",f_true(P)," ",p_true(P))

In [44]:
for i in range (4,-1,-1):
    print (i)

4
3
2
1
0
